### Environment setup and Log in to Twitter

In [12]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from time import sleep
import getpass

In [26]:
my_user = "madhu_2161"
my_pass = getpass.getpass()

In [28]:
search_item = "Liz Truss"

In [22]:
PATH = "C:\\Program Files\\chromedriver-win64\\chromedriver.exe"
service = Service(PATH)
driver = webdriver.Chrome(service=service)

driver.get("https://twitter.com/i/flow/login")
sleep(3)

In [24]:
user_id = driver.find_element(By.XPATH, "//Input[@type='text']")
user_id.send_keys(my_user)
user_id.send_keys(Keys.ENTER)

In [27]:
password = driver.find_element(By.XPATH, "//Input[@type='password']")
password.send_keys(my_pass)
password.send_keys(Keys.ENTER)

In [30]:
search_box = driver.find_element(By.XPATH, "//Input[@data-testid='SearchBox_Search_Input']")
search_box.send_keys(search_item)
search_box.send_keys(Keys.ENTER)

In [34]:
all_tweets = set()

tweets = driver.find_elements(By.XPATH, "//div[@data-testid='tweetText']")
while True:
    for tweet in tweets:
        all_tweets.add(tweet.text)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    sleep(3)  
    tweets = driver.find_elements(By.XPATH, "//div[@data-testid='tweetText']")  
    if len(all_tweets) > 50:
        break

In [35]:
all_tweets = list(all_tweets)
all_tweets[0]

'I think \n@RachelReevesMP\n will be popular... with Liz Truss ! The worst economy crashes in history are on the cards. \nRachel from the Call Centre I think.. not even Rachel from Accounts. \nThe Chancellor who had her Government credit card taken off her and claimed for expenses for'

### Cleaning the Tweets

In [41]:
import pandas as pd
import re
pd.options.display.max_colwidth = 1000

In [39]:
df = pd.DataFrame(all_tweets, columns=['tweets'])
df.head()

,tweets
0,I think \n@RachelReevesMP\n will be popular... with Liz Truss ! The worst economy crashes in history are on the cards. \nRachel from the Call Centre I think.. not even Rachel from Accounts. \nThe Chancellor who had her Government credit card taken off her and claimed for expenses for
1,The same Liz Truss that supported people with energy bills and capped them?\nThe same Liz Truss that reversed Sunak’s NI rise?\nThe same Liz Truss who was going to cut Income Tax for both basic and higher rate tax payers?\nThe same Liz Truss vilified for the BoE’s reckless QE?
2,How come Liz Truss’ line finishes so much sooner than anyone else’s?
3,"Remember his famous beef deal, the one that Liz Truss tried to reverse the next day. Australian Dan Tehan said your boss has conceded the whole kingdom I would have been sacked immediately if I had gone back with what Bunter had agreed too"
4,Ella es Liz Truss fue la primera liberal libertaria en ser primera ministra de Inglaterra en 2022.\n\nHizo un paquetazo de medidas liberales extremas al estilo Milei. Su gobierno duró solo 44 días . Estas ideas ya fracasaron.


In [44]:
one_tweet = df.iloc[0]['tweets']
one_tweet

'I think \n@RachelReevesMP\n will be popular... with Liz Truss ! The worst economy crashes in history are on the cards. \nRachel from the Call Centre I think.. not even Rachel from Accounts. \nThe Chancellor who had her Government credit card taken off her and claimed for expenses for'

In [45]:
cleanTweet =re.sub(r"@[a-zA-Z0-9]+", "", one_tweet)
cleanTweet =re.sub(r"#[a-zA-Z0-9\s]+", "", cleanTweet)
cleanTweet

'I think \n\n will be popular... with Liz Truss ! The worst economy crashes in history are on the cards. \nRachel from the Call Centre I think.. not even Rachel from Accounts. \nThe Chancellor who had her Government credit card taken off her and claimed for expenses for'